## 国会会議録のAPIからデータ取得する。
https://kokkai.ndl.go.jp/#/

- 第1回国会（昭和22年5月）からの本会議・委員会の会議録を検索し、閲覧できる。
- APIという仕組みを使えば、データを自動的に大量取得できる。
- まずAPIの仕様書を読みましょう。
- APIは、指定されたurlに検索条件を加えることで所望のデータを取得します。

https://kokkai.ndl.go.jp/api.html

- 1. アクセスURLの確認
    - 発言単位：http://kokkai.ndl.go.jp/api/1.0/speech?{検索条件}
    - 会議単位：http://kokkai.ndl.go.jp/api/1.0/meeting?{検索条件}
- 2. 検索条件の確認
- 3. 検索パラメータの確認
  - 	startRecord	開始位置	検索結果の取得開始位置を「1～検索件数」の範囲で指定可能。
  - 	maximumRecords	一回の最大取得件数	一回のリクエストで取得できるレコード数を、会議単位簡易出力
  - ...
- 4. 返戻結果の形式の確認
  - xml or json
  

### 準備
- 必要なモジュールをインストールしていきます。各モジュールの説明は使う場面で行います。


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/cssws2023/

Mounted at /content/drive
/content/drive/MyDrive/cssws2023


In [3]:
#xmlをdictに変換するモジュール
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import requests
import urllib.parse
import xmltodict 
import pprint
import pandas as pd
from collections import defaultdict

### APIからのデータ取得
- APIのurlとparamsを変数にまとめておく。
- fetchする関数を定義する。
- 1回の呼び出しで得られる件数が限られているため、'startsRecord'を指定して繰り返し取得する必要がある。


In [5]:
url = 'http://kokkai.ndl.go.jp/api/1.0/speech?'
params = {'any':'コロナ',
          'from':'2020-01-01',
          'until':'2023-04-13',
          'startRecord':'1'
}

In [6]:
def fetch_kokkaikaigiroku(url, params):
    dat = []
    while params['startRecord'] != None:#繰り返し取得する。
        print(f"{params['startRecord']}番からの取得を開始します。")
        response = requests.get(url,params)#requests.get()でデータ取得
        res = xmltodict.parse(response.content)
        dat.extend(res['data']['records']['record'])
        params['startRecord'] = res['data'].get('nextRecordPosition')
    return dat


In [7]:
dat = fetch_kokkaikaigiroku(url, params)

1番からの取得を開始します。
31番からの取得を開始します。
61番からの取得を開始します。
91番からの取得を開始します。
121番からの取得を開始します。
151番からの取得を開始します。
181番からの取得を開始します。
211番からの取得を開始します。
241番からの取得を開始します。
271番からの取得を開始します。
301番からの取得を開始します。
331番からの取得を開始します。
361番からの取得を開始します。
391番からの取得を開始します。
421番からの取得を開始します。
451番からの取得を開始します。
481番からの取得を開始します。
511番からの取得を開始します。
541番からの取得を開始します。
571番からの取得を開始します。
601番からの取得を開始します。
631番からの取得を開始します。
661番からの取得を開始します。
691番からの取得を開始します。
721番からの取得を開始します。
751番からの取得を開始します。
781番からの取得を開始します。
811番からの取得を開始します。
841番からの取得を開始します。
871番からの取得を開始します。
901番からの取得を開始します。
931番からの取得を開始します。
961番からの取得を開始します。
991番からの取得を開始します。
1021番からの取得を開始します。
1051番からの取得を開始します。
1081番からの取得を開始します。
1111番からの取得を開始します。
1141番からの取得を開始します。
1171番からの取得を開始します。
1201番からの取得を開始します。
1231番からの取得を開始します。
1261番からの取得を開始します。
1291番からの取得を開始します。
1321番からの取得を開始します。
1351番からの取得を開始します。
1381番からの取得を開始します。
1411番からの取得を開始します。
1441番からの取得を開始します。
1471番からの取得を開始します。
...
(出力をトリミングしました)

#### ディクショナリを整形してデータフレームに変換
- さらに、必要な項目だけを抜き出し、ディクショナリに整形しデータフレームに変換する
- defaultdict(list)はkey呼び出ししたときにキーがなければlistを作成する。
- GoogleDriveにマウントしてあれば、cssws2023の下にdataというディレクトリを作成し、そこにcsvファイルを保存する。

In [8]:
def to_DataFrame(dat):
    dat_dict = defaultdict(list)
    for item in dat:
        row = item['recordData']['speechRecord']#この階層に発言の実体がある。
        for key, value in row.items():
            dat_dict[key].append(value)
    df = pd.DataFrame(dat_dict)
    return df
    


In [9]:
df = to_DataFrame(dat)

In [10]:
import os 
if not os.path.exists('data'):
    os.makedirs('data')
df.to_csv('data/kokkai_corona.csv', index = False)

### APIからのデータ取得コードの補足説明
- #requests.get()でAPIのアクセスurlとパラメータを指定するとデータが取得できる。
- resonsen.contentにxml形式でテキストが格納される。
- xmltodict.parse()によってxmlをパースしてディクショナリに変換する。
- 発言内容は'data'>'records'>'record'という上から3階層目にある。
- 'data'>'nextRecordPosition'に次のスタート番号が記録されている。存在しない場合があるのでget()を使って取得。

In [ ]:
response = requests.get(url,params)

In [ ]:
response.content

b'<data><numberOfRecords>38917</numberOfRecords><numberOfReturn>30</numberOfReturn><startRecord>1</startRecord><nextRecordPosition>31</nextRecordPosition><records><record><recordData><speechRecord><speechID>121104376X01320230407_021</speechID><issueID>121104376X01320230407</issueID><imageKind>\xe4\xbc\x9a\xe8\xad\xb0\xe9\x8c\xb2</imageKind><searchObject>21</searchObject><session>211</session><nameOfHouse>\xe8\xa1\x86\xe8\xad\xb0\xe9\x99\xa2</nameOfHouse><nameOfMeeting>\xe8\xb2\xa1\xe5\x8b\x99\xe9\x87\x91\xe8\x9e\x8d\xe5\xa7\x94\xe5\x93\xa1\xe4\xbc\x9a</nameOfMeeting><issue>\xe7\xac\xac13\xe5\x8f\xb7</issue><date>2023-04-07</date><closing></closing><speechOrder>21</speechOrder><speaker>\xe5\xa4\xa7\xe5\x9d\xaa\xe5\xaf\x9b\xe5\xad\x90</speaker><speakerYomi></speakerYomi><speakerGroup></speakerGroup><speakerPosition>\xe5\x8e\x9a\xe7\x94\x9f\xe5\x8a\xb4\xe5\x83\x8d\xe7\x9c\x81\xe5\xa4\xa7\xe8\x87\xa3\xe5\xae\x98\xe6\x88\xbf\xe5\xaf\xa9\xe8\xad\xb0\xe5\xae\x98</speakerPosition><speakerRole>

In [ ]:
res = xmltodict.parse(response.content)

In [ ]:
res

{'data': {'numberOfRecords': '38917',
  'numberOfReturn': '30',
  'startRecord': '1',
  'nextRecordPosition': '31',
  'records': {'record': [{'recordData': {'speechRecord': {'speechID': '121104376X01320230407_021',
       'issueID': '121104376X01320230407',
       'imageKind': '会議録',
       'searchObject': '21',
       'session': '211',
       'nameOfHouse': '衆議院',
       'nameOfMeeting': '財務金融委員会',
       'issue': '第13号',
       'date': '2023-04-07',
       'closing': None,
       'speechOrder': '21',
       'speaker': '大坪寛子',
       'speakerYomi': None,
       'speakerGroup': None,
       'speakerPosition': '厚生労働省大臣官房審議官',
       'speakerRole': None,
       'speech': '○大坪政府参考人\u3000お答え申し上げます。\n\u3000国立病院機構でございますけれども、これは全国に百四十の病院がございます、また、地域医療機能推進機構、これは全国に五十七の病院がございまして、地域医療や国の政策医療でございます五疾病五事業、こういった医療の提供のほか、他の設置主体では必ずしも実施されないおそれのある医療の提供などを行う独立行政法人でございます。\n\u3000この両法人におきましては、これまで新型コロナの病床の確保、また臨時の医療施設への人材派遣、こういったことでも、通常の診療を行いつつ御尽力をいただいたところでございます。\n\u3000今般の国庫の納付でありますけれども、政府の方針として、防衛力を抜本的

In [ ]:
res['data']['records']['record']

[{'recordData': {'speechRecord': {'speechID': '121104376X01320230407_021',
    'issueID': '121104376X01320230407',
    'imageKind': '会議録',
    'searchObject': '21',
    'session': '211',
    'nameOfHouse': '衆議院',
    'nameOfMeeting': '財務金融委員会',
    'issue': '第13号',
    'date': '2023-04-07',
    'closing': None,
    'speechOrder': '21',
    'speaker': '大坪寛子',
    'speakerYomi': None,
    'speakerGroup': None,
    'speakerPosition': '厚生労働省大臣官房審議官',
    'speakerRole': None,
    'speech': '○大坪政府参考人\u3000お答え申し上げます。\n\u3000国立病院機構でございますけれども、これは全国に百四十の病院がございます、また、地域医療機能推進機構、これは全国に五十七の病院がございまして、地域医療や国の政策医療でございます五疾病五事業、こういった医療の提供のほか、他の設置主体では必ずしも実施されないおそれのある医療の提供などを行う独立行政法人でございます。\n\u3000この両法人におきましては、これまで新型コロナの病床の確保、また臨時の医療施設への人材派遣、こういったことでも、通常の診療を行いつつ御尽力をいただいたところでございます。\n\u3000今般の国庫の納付でありますけれども、政府の方針として、防衛力を抜本的に強化していくための財源、これについて税外収入の確保に最大限取り組むといった方針の中で、それぞれの法人の個別法に基づきまして、中期計画期間満了時に、元々、次期計画中に必要な業務の財源に充てるために繰越しが認められた額を除いて、国庫に納付することとなっております。\n\u3000こういった規定を踏まえて、新型コロナ対策の予算等によって積み上がっております積立金、これのうち約〇

### 課題
- 自分の興味のあるテーマで検索し、データを取得してください。
- ただし、あまりにデータが多くなると取得に時間がかかるので、最大2万件程度を目安にしてください。
